In [1]:
import numpy as np
import pandas as pd
import sys 
eps = np.finfo(float).eps
from numpy import log2 as log
import random
from pprint import pprint

In [2]:
attributes = {
    'A1': ['b','a'],
    'A2': 'continuous',
    'A3': 'continuous',
    'A4': ['u','y','l','t'],
    'A5': ['g','p','gg'],
    'A6': ['c','d','cc','i','j','k','m','r','q','w','x','e','aa','ff'],
    'A7': ['v','h','bb','j','n','z','dd','ff','o'],
    'A8': 'continuous',
    'A9': ['t','f'],
    'A10': ['t','f'],
    'A11': 'continuous',
    'A12': ['t','f'],
    'A13': ['g','p','s'],
    'A14': 'continuous',
    'A15': 'continuous',
    'label': ['+','-'],
}

In [3]:
train_df= pd.read_csv("train.txt",sep = '\t', header = None, names = attributes.keys())
train_df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,label
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202,0,+
1,a,24.5,0.500,u,g,q,h,1.50,t,f,0,f,g,280,824,+
2,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120,0,+
3,b,32.08,4.000,u,g,m,v,2.50,t,f,0,t,g,360,0,+
4,a,45.83,10.500,u,g,q,v,5.00,t,t,7,t,g,0,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,260,0,-
486,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,200,394,-
487,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,200,1,-
488,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,280,750,-


In [4]:
def fill_missing_values(df):
    for key in df.keys()[:-1]:
        df[key] = df[key].replace('?',df[key].mode()[0])
        
fill_missing_values(train_df)

In [5]:
train_data = train_df.values
train_data[:5]

array([['b', '30.83', 0.0, 'u', 'g', 'w', 'v', 1.25, 't', 't', 1, 'f',
        'g', '202', 0, '+'],
       ['a', '24.5', 0.5, 'u', 'g', 'q', 'h', 1.5, 't', 'f', 0, 'f', 'g',
        '280', 824, '+'],
       ['b', '20.17', 5.625, 'u', 'g', 'w', 'v', 1.71, 't', 'f', 0, 'f',
        's', '120', 0, '+'],
       ['b', '32.08', 4.0, 'u', 'g', 'm', 'v', 2.5, 't', 'f', 0, 't',
        'g', '360', 0, '+'],
       ['a', '45.83', 10.5, 'u', 'g', 'q', 'v', 5.0, 't', 't', 7, 't',
        'g', '0', 0, '+']], dtype=object)

In [6]:
def check_label_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [7]:
def majority_class(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [8]:
def get_potential_splits(data):
    
    potential_splits = {}
    number_of_columns = data.shape[1]
    for column_index in range(number_of_columns - 1):          # excluding the target column
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

In [9]:
def split_data(data, split_column, split_value):
    
    split_column_data = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "categorical": 
        data_above_split = data[split_column_data != split_value]
        data_below_split = data[split_column_data == split_value]
        
    # feature is continuous   
    else:
        data_above_split = data[split_column_data >  split_value]
        data_below_split = data[split_column_data <= split_value]
    
    return data_above_split, data_below_split
# data_below, data_above = split_data(data, best_split_column, best_split_value)
# data_below, data_above

In [10]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy
# calculate_entropy(data)

In [11]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [12]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_above, data_below = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value
# best_split_column, best_split_value = determine_best_split(data,pot)
# best_split_column, best_split_value

In [13]:
sub_tree = {"question": ["yes_answer", 
                         "no_answer"]}

In [14]:
def determine_type_of_feature(df):
    feature_types = []
    n_unique_values_treshold = 14
    for feature in df.columns:
        print(feature)
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
                print('c')
            else:
                feature_types.append("continuous")
                print('conti')
    
    return feature_types
# determine_type_of_feature(train_df)

In [15]:
def decision_tree_algorithm(df, max_depth, min_samples=2, counter=0):
    
    # data preparations
    if counter == 0:
        global COLUMN_LABELS, FEATURE_TYPES
        COLUMN_LABELS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_label_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = majority_class(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_above, data_below = split_data(data, split_column, split_value)
        
        # check for empty data
        if len(data_below) == 0:
            classification = majority_class(data)
            return classification
        
        if len(data_above) == 0:
            classification = majority_class(data)
            return classification
        
        # determine question
        feature_name = COLUMN_LABELS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "categorical":
            question = "{} = {}".format(feature_name, split_value)
            
        # feature is continuous
        else:
            question = "{} <= {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, max_depth, min_samples, counter)
        no_answer = decision_tree_algorithm(data_above, max_depth, min_samples, counter)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [16]:
def decision_tree_algorithm_without_depth(df, min_samples=2, counter=0):
    
    # data preparations
    if counter == 0:
        global COLUMN_LABELS, FEATURE_TYPES
        COLUMN_LABELS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_label_purity(data)) or (len(data) < min_samples):
        classification = majority_class(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_above, data_below = split_data(data, split_column, split_value)
        
        # check for empty data
        if len(data_below) == 0:
            classification = majority_class(data)
            return classification
        
        if len(data_above) == 0:
            classification = majority_class(data)
            return classification
        
        # determine question
        feature_name = COLUMN_LABELS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "categorical":
            question = "{} = {}".format(feature_name, split_value)
            
        # feature is continuous
        else:
            question = "{} <= {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm_without_depth(data_below, min_samples, counter)
        no_answer = decision_tree_algorithm_without_depth(data_above, min_samples, counter)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [17]:
tree = decision_tree_algorithm(train_df, max_depth=6)
pprint(tree)

A1
c
A2
conti
A3
conti
A4
c
A5
c
A6
c
A7
c
A8
conti
A9
c
A10
c
A11
conti
A12
c
A13
c
A14
conti
A15
conti
label
{'A9 = t': [{'A15 <= 375': [{'A14 <= 300': [{'A8 <= 1.375': [{'A2 <= 19.75': ['-',
                                                                              '+']},
                                                             {'A5 = p': [{'A11 <= 6': ['-',
                                                                                       '+']},
                                                                         {'A14 <= 30': ['+',
                                                                                        '-']}]}]},
                                            '+']},
                            '+']},
            {'A3 <= 0.165': [{'A2 <= 22.5': [{'A7 = n': ['+', '-']},
                                             {'A6 = c': [{'A2 <= 25.58': ['+',
                                                                          {'A2 <= 34.58': ['-',
             

In [21]:
tree_without_depth = decision_tree_algorithm_without_depth(train_df)
pprint(tree_without_depth)

A1
c
A2
conti
A3
conti
A4
c
A5
c
A6
c
A7
c
A8
conti
A9
c
A10
c
A11
conti
A12
c
A13
c
A14
conti
A15
conti
label
{'A9 = t': [{'A15 <= 375': [{'A14 <= 300': [{'A8 <= 1.375': [{'A2 <= 19.75': ['-',
                                                                              {'A8 <= 0.46': [{'A3 <= 11.5': ['+',
                                                                                                              {'A2 <= 54.83': ['-',
                                                                                                                               '+']}]},
                                                                                              {'A8 <= 0.54': ['-',
                                                                                                              {'A8 <= 0.835': ['+',
                                                                                                                               {'A11 <= 3': [{'A12 = t': ['-',
                 

In [18]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":  # feature is continuous
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [19]:
example = train_df.iloc[150]
example

A1            b
A2        38.17
A3       10.125
A4            u
A5            g
A6            x
A7            v
A8          2.5
A9            t
A10           t
A11           6
A12           f
A13           g
A14         520
A15         196
label         +
Name: 150, dtype: object

In [22]:
classify_example(example, tree_without_depth)

TypeError: '<=' not supported between instances of 'str' and 'float'

In [23]:
def get_predicted_labels(df):
    predict_labels = []
    for i in range(len(df.index)):
        inst = df.iloc[i]
#         prediction = classify_example(inst, tree)
        predict_labels.append(classify_example(inst, tree))
    original_labels = df['label'].values
    predict_labels = np.array(predict_labels)
    return predict_labels, original_labels

In [24]:
predict,original = get_predicted_labels(train_df)
# print(predict,original)

TypeError: '<=' not supported between instances of 'str' and 'float'

In [37]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [38]:
accuracy = calculate_accuracy(train_df, tree)
accuracy

TypeError: '<=' not supported between instances of 'str' and 'float'

In [164]:
test_df= pd.read_csv("test.txt",sep = '\t', header = None, names = attributes.keys())
test_df
fill_missing_values(test_df)

In [165]:
test_accuracy = calculate_accuracy(test_df, tree)
test_accuracy

0.6044776119402985